In [34]:
import pandas as pd
import urllib.request
from bs4 import BeautifulSoup

In [35]:
def saveInBase(data, name):
    import sqlite3
    conn = sqlite3.connect(name)
    data.to_sql('bae', conn)
    conn.close()

In [36]:
def parsePageInMusic(soup):
    """
    parse page
    :param: input soup object
    :return list
    """
    result = list()
    tables = soup.find_all('table', { 'class':"wikitable",'width':'100%'}) 
    for table in tables:
        rows = table.findChildren('tr')
        month = ''
        day = None
        for row in rows:  
            cells = row.findChildren('td')
            album = None
            artist = None
            notes = None
            if len(cells) == 5:
                month = ('').join(cells[0].b.contents[::2]).lower().title()
                day = int(cells[1].string)
                album = cells[2].string
                artist = cells[3].string
                notes = cells[4].string
            elif len(cells) == 4:
                try:
                    day = int(cells[0].string)
                except (ValueError, TypeError):
                    day = day
                
                album = cells[1].string
                artist = cells[2].string
                notes = cells[3].string
            elif len(cells) == 3:
                album = cells[0].string
                artist = cells[1].string
                notes = cells[2].string            

            else:
                pass
            
            result.append(dict(month=month, day=day,album=album,artist=artist,notes=notes))
        
    return result       

In [37]:
def downloadPage(url):
    """
    download raw page
    :return soup object
    """
    with urllib.request.urlopen(url) as response:
        html = response.read()

    return BeautifulSoup(html)

In [38]:
urls = [ 'https://en.wikipedia.org/wiki/{}_in_music'.format(i) for i in range(1990,2000)]
table = list()
for url in urls:
    soup = downloadPage(url)
    table.extend(parsePageInMusic(soup))

/usr/local/lib/python3.5/dist-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file /usr/lib/python3.5/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html5lib")

  markup_type=markup_type))


In [39]:
## create data frame and save to base
data = pd.DataFrame(table)
saveInBase(data, 'baseq3_1.sqlite.db')

In [40]:
u = 'https://en.m.wikipedia.org/wiki/List_of_best-selling_music_artists'
s = downloadPage(u)

/usr/local/lib/python3.5/dist-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file /usr/lib/python3.5/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html5lib")

  markup_type=markup_type))


In [41]:
def parseBestSalesMusic(soup):
    """
    parse https://en.m.wikipedia.org/wiki/List_of_best-selling_music_artists
    """
    result = list()
    tables = s.find_all('table')
    for table in tables:
        for i in table.findChildren('tr'):
            td = i.findChildren('td')
            if td:
                country = td[0].string
                group = td[1].sup.a.attrs['href'].split(':')[1]
                genre = td[3].a.text
                total = td[4].text
                sales = td[5].text
                result.append(dict(country=country, group=group, genre=genre, total=total, sales=sales))
                
    return result

In [42]:
d = parseBestSalesMusic(s)
data_best_music = pd.DataFrame(d)
saveInBase(data_best_music,'base_best_music.sqlite.db')